In [1]:
!wget https://www.dropbox.com/s/viz3bmc8cil4w1y/train_data.csv?dl=1
!wget https://www.dropbox.com/s/07wu5by7llczd36/test_data.csv?dl=1
!wget https://www.dropbox.com/s/meymqbp1wrl23q6/sample_solution.csv?dl=1
!pip install gensim
import gensim.downloader as api
word_vectors = api.load('glove-wiki-gigaword-100') 

--2023-04-19 04:26:33--  https://www.dropbox.com/s/viz3bmc8cil4w1y/train_data.csv?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/viz3bmc8cil4w1y/train_data.csv [following]
--2023-04-19 04:26:33--  https://www.dropbox.com/s/dl/viz3bmc8cil4w1y/train_data.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc0073fd01b38389862667c68948.dl.dropboxusercontent.com/cd/0/get/B6e9fZ7ZLmbDMgW3_1llinDjE2wazkLuJEhbXcSTZpLfPsBdXUNMPjcSxBnDZaAELdeHqZGYofuXjgp-HOxfex-azro9iY9qZN8adG9oyrwkxl0n14saUbg8jv5u8leuB1Gq8mhaYfAHgkJc0qxJoMp1j84xclHoOH77UI3dSk108Q/file?dl=1# [following]
--2023-04-19 04:26:34--  https://uc0073fd01b38389862667c68948.dl.dropboxusercontent.com/cd/0/get/B6e9fZ7ZLmbDMgW3_1llinDjE2wazkLuJEhbXcSTZpLfPsBdXUNMPjcSxBnDZaAELdeHqZGY

In [2]:
import pandas as pd
import numpy as np
train_data = pd.read_csv('train_data.csv?dl=1', encoding = 'latin-1')
test_data = pd.read_csv('test_data.csv?dl=1', encoding = 'latin-1')
train_data.iloc[0:5]

## Tokenize ##
from tensorflow.keras.preprocessing.text import Tokenizer
texts = train_data['Text']
test_sentences = test_data['Text']
labels = train_data['Label']

max_words = 10000
token = Tokenizer(max_words)
token.fit_on_texts(texts)
vocab_size = max_words + 1

sequences = token.texts_to_sequences(texts)
test_sequences = token.texts_to_sequences(test_sentences)

## Convert to sequences ##
from tensorflow.keras.preprocessing.sequence import pad_sequences
seq_len = 125
X = pad_sequences(sequences, maxlen=seq_len)
X_test = pad_sequences(test_sequences, maxlen=seq_len)

## Make y one-hot ##
y = np.zeros( (len(labels),len(np.unique(labels)) ) )
for l in np.unique(labels):
    pos_inds = np.where(labels == l)[0]
    y[pos_inds,l] = 1

num_classes = y.shape[1]


In [3]:
embeddings_index = {}
vocab = token.word_index.keys()
for word in vocab:
    if word in word_vectors.index_to_key:
      coefs = np.asarray(word_vectors[word], dtype='float32')
      embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

word_index = token.word_index
embedding_matrix = np.zeros( (vocab_size , 100) )
for word, i in word_index.items():
    if i < vocab_size:
      embedding_vector = embeddings_index.get(word)
      if embedding_vector is not None:
          # words not found in embedding index will be all-zeros.
          embedding_matrix[i] = embedding_vector

Found 56914 word vectors.


In [4]:
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D, Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding, Flatten
from tensorflow.keras.models import Model

input = Input(shape=(seq_len,), dtype='int32')
x = Embedding(vocab_size, 
              output_dim = 100,
              weights=[embedding_matrix],
              input_length=seq_len)(input)

x = Conv1D(filters=128, kernel_size=5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)

x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
preds = Dense(num_classes, activation='softmax')(x)

model = Model(input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

model.fit(X, y,
          batch_size=128,
          epochs=10,
          validation_split=0.2)


Epoch 1/10
71/71 [==============================] - 27s 225ms/step - loss: 2.9266 - acc: 0.0942 - val_loss: 2.6299 - val_acc: 0.1993
Epoch 2/10
71/71 [==============================] - 9s 131ms/step - loss: 2.3731 - acc: 0.2166 - val_loss: 1.9668 - val_acc: 0.3221
Epoch 3/10
71/71 [==============================] - 9s 123ms/step - loss: 1.8228 - acc: 0.3783 - val_loss: 1.4617 - val_acc: 0.5400
Epoch 4/10
71/71 [==============================] - 6s 90ms/step - loss: 1.3640 - acc: 0.5359 - val_loss: 1.1678 - val_acc: 0.6226
Epoch 5/10
71/71 [==============================] - 5s 70ms/step - loss: 1.0305 - acc: 0.6566 - val_loss: 1.0386 - val_acc: 0.6518
Epoch 6/10
71/71 [==============================] - 4s 60ms/step - loss: 0.8170 - acc: 0.7309 - val_loss: 0.9351 - val_acc: 0.6938
Epoch 7/10
71/71 [==============================] - 5s 66ms/step - loss: 0.6214 - acc: 0.7942 - val_loss: 0.9138 - val_acc: 0.7141
Epoch 8/10
71/71 [==============================] - 4s 64ms/step - loss: 0.4852

In [5]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 125)]             0         
                                                                 
 embedding (Embedding)       (None, 125, 100)          1000100   
                                                                 
 conv1d (Conv1D)             (None, 121, 128)          64128     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 24, 128)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 20, 128)           82048     
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                              

In [6]:
test_preds = model.predict(X_test, batch_size=256)
submission_template = pd.read_csv('sample_solution.csv?dl=1')

for j in range(test_preds.shape[1]):
  submission_template.iloc[:,j+1] = test_preds[:,j]

submission_template.to_csv('submission.csv', index=False)

from google.colab import files
files.download('submission.csv')

30/30 [==============================] - 1s 8ms/step


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>